In [1]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from typing import List

from loguru import logger

os.chdir("..")

In [2]:
# Config

current_year = 2026

suffix_current_year = f"{current_year-1}_{str(current_year)[-2:]}"

DATA_DIR = "data"
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")

NUM_TEAMS = 3  # Squadre top da costruire

In [3]:
def train_top_player_model(df: pd.DataFrame, suffix_cy: str):
    """Addestra un modello Random Forest per prevedere i "Fenomeni" e 
    restituisce il modello e l'importanza delle feature.

    Args:
        df (pd.DataFrame): Il DataFrame contenente i dati dei giocatori.

    Returns:
        tuple: Il modello addestrato, un DataFrame con l'importanza delle feature, 
               il DataFrame pulito e la lista delle colonne delle feature usate.
    """
    # Creiamo una copia del dataframe per non modificare l'originale
    df_temp = df.copy()

    # Definiamo la variabile target e le feature
    target_column = "Fenomeno"
    
    # Selezioniamo le feature. Escludiamo le colonne che non servono o che sono la target stessa.
    feature_columns = [
        col for col in df_temp.columns 
        if col not in [
            "Id", 
            "Nome", 
            f"R_{suffix_cy}", 
            "Squadra_{suffix_cy}", 
            target_column, 
            "Affarone", 
            "Punteggio_Performance_Ponderato", 
            "Punteggio_Performance_Ponderato_Normalizzato", 
            "Media_Fantamedia_Ponderata", 
            "Media_Partite_Giocate_Ponderata"
        ] and df_temp[col].dtype in ["int64", "float64"]
    ]
    
    # Rimuoviamo le righe dove la variabile target è mancante
    cleaned_df = df_temp.dropna(subset=[target_column] + feature_columns)

    if cleaned_df.empty:
        logger.error("Il dataset è vuoto dopo la pulizia. Impossibile addestrare il modello.")
        return None, None, None, None

    # Assegnamo le feature (X) e la variabile target (y)
    X = cleaned_df[feature_columns]
    y = cleaned_df[target_column]

    # Controlliamo la distribuzione della variabile target
    if y.nunique() < 2:
        logger.error(f"La variabile target '{target_column}' ha meno di due classi ({y.nunique()}). Impossibile addestrare un classificatore.")
        return None, None, None, None

    # Suddividiamo i dati in set di addestramento e di test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    logger.info(f"Dimensioni del set di addestramento: {X_train.shape[0]} righe")
    logger.info(f"Dimensioni del set di test: {X_test.shape[0]} righe")
    logger.info(f"Distribuzione del target nel set di addestramento:\n{y_train.value_counts(normalize=True)}")

    # Creazione del modello Random Forest
    model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
    logger.info("Modello Random Forest creato. Avvio l'addestramento...")

    # Addestramento del modello
    model.fit(X_train, y_train)


    # Valutazione del modello sul set di test
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    logger.info(f"\nAccuratezza del modello: {accuracy:.2f}")
    logger.info("\nReport di classificazione:")
    logger.info(report)

    # Mostra l'importanza delle feature
    feature_importances = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    logger.info("\nImportanza delle feature:")
    logger.info(feature_importances.head(15).to_string(index=False))

    logger.success("--- Addestramento completato! ---")
    return model, feature_importances, cleaned_df, feature_columns

In [4]:
def classify_players_with_ai(df: pd.DataFrame, model: RandomForestClassifier, feature_columns: List[str], suffix_cy: str):
    """
    Usa il modello addestrato per classificare e ordinare i giocatori, aggiungendo
    le nuove colonne AI al DataFrame.

    Args:
        df (pd.DataFrame): Il DataFrame dei giocatori.
        model: Il modello Random Forest addestrato.
        feature_columns (list): Le colonne delle feature usate per l'addestramento.

    Returns:
        pd.DataFrame: Un DataFrame con le nuove colonne AI aggiunte.
    """

    # Creiamo una copia del DataFrame per non alterare l'originale
    df_processed = df.copy()
    
    # Rimuoviamo le colonne non numeriche dal set di feature per la previsione
    numeric_feature_columns = [col for col in feature_columns if df_processed[col].dtype in ['int64', 'float64']]
    
    # Gestione dei valori mancanti nelle feature
    df_to_classify = df_processed.dropna(subset=numeric_feature_columns)

    if df_to_classify.empty:
        logger.error("Nessun giocatore con dati completi per la classificazione.")
        return df_processed
    
    # Assicuriamo l'ordine corretto delle colonne per la previsione
    X_predict = df_to_classify[numeric_feature_columns]

    # Calcola le probabilità di essere un Fenomeno
    probabilities = model.predict_proba(X_predict)[:, 1]
    
    # Aggiungi le nuove colonne AI al DataFrame temporaneo
    df_to_classify['Probabilità_Fenomeno'] = probabilities
    df_to_classify['Fenomeno_AI'] = (probabilities > 0.5).astype(int)
    
    # L'Indice Beccalossi AI si basa sempre sul prodotto tra il vecchio fattore e la probabilità del modello
    df_to_classify['Indice_Beccalossi_AI'] = df_to_classify['Indice_Beccalossi'] * probabilities
    
    # Un affarone è un giocatore a basso costo con un alto Indice Beccalossi AI.
    low_price_threshold = df_to_classify[f"Qt.A_{suffix_cy}"].quantile(0.50) if not df_to_classify[f"Qt.A_{suffix_cy}"].dropna().empty else 10
    
    valid_becca_index_ai = df_to_classify['Indice_Beccalossi_AI'][df_to_classify['Indice_Beccalossi_AI'] > 0]
    high_performance_threshold_ai = valid_becca_index_ai.quantile(0.75) if not valid_becca_index_ai.empty else 0.1

    logger.info(f"Soglia prezzo basso (Qt.A_{suffix_cy}): <= {low_price_threshold:.2f}")
    logger.info(f"Soglia performance alta (Indice_Beccalossi_AI): >= {high_performance_threshold_ai:.2f}")

    df_to_classify['Affarone_AI'] = 0
    goldenboy_condition = (df_to_classify[f"Qt.A_{suffix_cy}"].notna()) & (df_to_classify[f"Qt.A_{suffix_cy}"] <= low_price_threshold) & (df_to_classify[f"Qt.A_{suffix_cy}"] > 0) & \
                        (df_to_classify['Indice_Beccalossi_AI'] >= high_performance_threshold_ai) & (df_to_classify['Indice_Beccalossi_AI'].notna())

    df_to_classify.loc[goldenboy_condition, 'Affarone_AI'] = 1

    # Unisci i risultati al DataFrame originale
    df_processed = df_processed.set_index('Id')
    df_to_classify = df_to_classify.set_index('Id')
    
    # Devi includere 'Probabilità_Fenomeno' qui
    df_processed['Probabilità_Fenomeno'] = df_to_classify['Probabilità_Fenomeno']
    df_processed['Fenomeno_AI'] = df_to_classify['Fenomeno_AI']
    df_processed['Indice_Beccalossi_AI'] = df_to_classify['Indice_Beccalossi_AI']
    df_processed['Affarone_AI'] = df_to_classify['Affarone_AI']
    
    # Riempi i valori mancanti con 0
    df_processed[['Probabilità_Fenomeno', 'Fenomeno_AI', 'Indice_Beccalossi_AI', 'Affarone_AI']] = df_processed[['Probabilità_Fenomeno', 'Fenomeno_AI', 'Indice_Beccalossi_AI', 'Affarone_AI']].fillna(0)

    logger.success("--- Classificazione completata con successo! ---")
    return df_processed.reset_index()

In [5]:
def build_ai_teams(df: pd.DataFrame, suffix_cy: int, num_teams: int = 3):
    """
    Crea N squadre basate sulla probabilità di essere un Fenomeno,
    con una logica avanzata per la selezione dei portieri.

    Args:
        df (pd.DataFrame): Il DataFrame dei giocatori con le probabilità AI.
        num_teams (int): Il numero di squadre da generare.

    Returns:
        pd.DataFrame: Il DataFrame dei giocatori con la colonna 'Squadra_AI' aggiunta.
    """
    # Assicuriamo che esista il ruolo per l'anno corrente
    if f"R_{suffix_cy}" not in df.columns:
        logger.error(f"Attenzione: la colonna 'R_{suffix_cy}' non è presente. Impossibile creare le squadre per ruolo.")
        return df, {}
    
    teams = {f'Squadra_{i+1}_AI': [] for i in range(num_teams)}
    ruoli_count = {'P': 3, 'D': 8, 'C': 8, 'A': 6}
    
    df_disponibili = df.copy()

    # --- NUOVA LOGICA PER I PORTIERI ---
    logger.info("Inizio la selezione dei portieri con la nuova logica...")
    
    portieri_df = df_disponibili[df_disponibili[f"R_{suffix_cy}"] == 'P'].sort_values('Probabilità_Fenomeno', ascending=False)
    
    # Definisci la soglia per i portieri titolari. Usiamo la media ponderata delle partite giocate.
    # Calcoliamo la soglia come il 50 percentile (mediana) tra i portieri disponibili.
    pv_col = 'Media_Partite_Giocate_Ponderata'
    if pv_col not in portieri_df.columns or portieri_df[pv_col].dropna().empty:
        # Fallback se la colonna non esiste o è vuota
        logger.warning(f"Attenzione: la colonna '{pv_col}' non è valida. Uso una soglia fissa di 19 partite.")
        pv_threshold = 19
    else:
        pv_threshold = portieri_df[pv_col].quantile(0.50)
        logger.info(f"Soglia Partite Giocate Ponderate per portieri: > {pv_threshold:.2f}")

    portieri_titolari = portieri_df[portieri_df[pv_col] >= pv_threshold].sort_values('Probabilità_Fenomeno', ascending=False)
    portieri_riserve = portieri_df[portieri_df[pv_col] < pv_threshold].sort_values('Probabilità_Fenomeno', ascending=False)
    
    # Assegna un portiere titolare a ogni squadra
    titolari_assegnati_id = []
    for team_id in teams:
        if not portieri_titolari.empty:
            portiere_scelto = portieri_titolari.iloc[0]
            teams[team_id].append(portiere_scelto['Id'])
            titolari_assegnati_id.append(portiere_scelto['Id'])
            # Rimuovi il portiere scelto dall'elenco dei disponibili
            portieri_titolari = portieri_titolari.drop(portiere_scelto.name)
        else:
            logger.warning(f"Attenzione: non ci sono abbastanza portieri 'titolari' da assegnare a tutte le squadre. Passaggio a logica di riserva.")
            break
            
    # Combina i portieri rimanenti per l'assegnazione successiva
    portieri_rimanenti = pd.concat([portieri_titolari, portieri_riserve]).sort_values('Probabilità_Fenomeno', ascending=False)
    
    # Assegna i restanti 2 portieri per ogni squadra
    for team_id in teams:
        num_portieri_attuali = len(teams[team_id])
        while num_portieri_attuali < ruoli_count['P']:
            if not portieri_rimanenti.empty:
                portiere_scelto = portieri_rimanenti.iloc[0]
                teams[team_id].append(portiere_scelto['Id'])
                # Rimuovi il portiere scelto dall'elenco dei disponibili
                portieri_rimanenti = portieri_rimanenti.drop(portiere_scelto.name)
                num_portieri_attuali += 1
            else:
                logger.warning("Attenzione: non ci sono più portieri disponibili per completare le squadre.")
                break

    # Rimuovi i portieri selezionati dal DataFrame generale dei disponibili
    all_selected_p_ids = [pid for p_list in teams.values() for pid in p_list]
    df_disponibili = df_disponibili[~df_disponibili['Id'].isin(all_selected_p_ids)]
    
    logger.info("Selezione dei portieri completata. Procedo con gli altri ruoli.")
    
    # --- FINE NUOVA LOGICA PER I PORTIERI ---
    
    # Selezioniamo gli altri ruoli (la logica rimane la stessa)
    for ruolo, count in ruoli_count.items():
        if ruolo == 'P': continue
        
        giocatori_disponibili_ruolo = df_disponibili[
            (df_disponibili[f"R_{suffix_cy}"] == ruolo)
        ].sort_values('Probabilità_Fenomeno', ascending=False)
        
        if not giocatori_disponibili_ruolo.empty:
            for i in range(count * num_teams):
                idx = i % len(giocatori_disponibili_ruolo)
                team_idx = i % num_teams
                
                player_id = giocatori_disponibili_ruolo.iloc[idx]['Id']
                
                teams[f'Squadra_{team_idx+1}_AI'].append(player_id)
                df_disponibili = df_disponibili.drop(giocatori_disponibili_ruolo.iloc[idx].name, errors='ignore')

    # Aggiungi la colonna Squadra_AI al DataFrame
    df['Squadra_AI'] = 'N.D.'
    for team_id, player_ids in teams.items():
        df.loc[df['Id'].isin(player_ids), 'Squadra_AI'] = team_id
        
    logger.success(f"--- Creazione di {num_teams} sqadre completata! ---")
    return df, teams

In [6]:
loaded_dataset = pd.read_csv(os.path.join(PROCESSED_DIR,"Dataset_processed_full.bcsv"))

In [ ]:
if not loaded_dataset.empty:
    logger.info("\n--- Inizio l'addestramento del modello Random Forest per i Fenomeni ---")
    top_player_model, feature_importances, model_df, model_feature_columns = train_top_player_model(loaded_dataset, suffix_current_year)
    
    if top_player_model is not None:
        logger.info("\n--- Classificazione dei giocatori per probabilità di essere 'Fenomeno' ---")
        loaded_dataset = classify_players_with_ai(
            df=loaded_dataset,
            model=top_player_model,
            feature_columns=model_feature_columns,
            suffix_cy=suffix_current_year,
        )
        
        logger.info(f"\n--- Creazione di {NUM_TEAMS} squadre AI ideali ---")
        loaded_dataset, ai_teams = build_ai_teams(loaded_dataset, suffix_current_year, NUM_TEAMS)
        
        final_output_file = "Database.csv"
        loaded_dataset.to_csv(final_output_file, index=False)
        logger.success(f"Il dataset unificato, inclusi i calcoli AI, è stato salvato in '{final_output_file}'")
        logger.info(f"Dimensioni del dataset finale: {loaded_dataset.shape[0]} righe, {loaded_dataset.shape[1]} colonne.")
    else:
        logger.warning
else:
    logger.error("Nessun dataset finale da salvare.")
        

2025-08-30 13:06:36.470 | INFO     | __main__:<module>:2 - 
--- Inizio l'addestramento del modello Random Forest per i Fenomeni ---
2025-08-30 13:06:36.478 | INFO     | __main__:train_top_player_model:54 - Dimensioni del set di addestramento: 873 righe
2025-08-30 13:06:36.478 | INFO     | __main__:train_top_player_model:55 - Dimensioni del set di test: 219 righe
2025-08-30 13:06:36.480 | INFO     | __main__:train_top_player_model:56 - Distribuzione del target nel set di addestramento:
Fenomeno
0    0.846506
1    0.153494
Name: proportion, dtype: float64
2025-08-30 13:06:36.481 | INFO     | __main__:train_top_player_model:60 - Modello Random Forest creato. Avvio l'addestramento...
2025-08-30 13:06:36.736 | INFO     | __main__:train_top_player_model:71 - 
Accuratezza del modello: 0.97
2025-08-30 13:06:36.737 | INFO     | __main__:train_top_player_model:72 - 
Report di classificazione:
2025-08-30 13:06:36.737 | INFO     | __main__:train_top_player_model:73 -               precision    rec

In [ ]:
# Stampa dei risultati
logger.info("\n=== I migliori giocatori scelti dal modello (indipendentemente dalla squadra) ===")

display_columns_ai = ['Nome', f"R_{suffix_current_year}", f"Squadra_{suffix_current_year}", f"Qt.A_{suffix_current_year}", 'Indice_Beccalossi_AI', 'Fenomeno_AI', 'Affarone_AI']
top_players_ai = loaded_dataset.sort_values(by='Indice_Beccalossi_AI', ascending=False)
logger.info(top_players_ai[display_columns_ai].head(10).to_string(index=False))
print()
logger.info("=== Dettagli delle squadre AI generate ===")
for team_name, player_ids in ai_teams.items():
    team_df = loaded_dataset[loaded_dataset['Id'].isin(player_ids)].sort_values(by=[f"R_{suffix_current_year}", f"Qt.A_{suffix_current_year}"])
    logger.info(f"\nSquadra: {team_name}")
    logger.info(team_df[['Nome', f"R_{suffix_current_year}", f"Squadra_{suffix_current_year}", f"Qt.A_{suffix_current_year}"]].to_string(index=False))

2025-08-30 13:11:48.671 | INFO     | __main__:<module>:2 - 
=== I 3 migliori giocatori scelti dal modello (indipendentemente dalla squadra) ===
2025-08-30 13:11:48.680 | INFO     | __main__:<module>:6 -        Nome R_2025_26 Squadra_2025_26  Qt.A_2025_26  Indice_Beccalossi_AI  Fenomeno_AI  Affarone_AI
    Darmian         D           Inter             4              0.408772            1            1
    Luperto         D        Cagliari             9              0.376078            1            1
      Zappa         D        Cagliari             6              0.361580            1            1
Baschirotto         D       Cremonese            10              0.357605            1            1
 Mkhitaryan         C           Inter             9              0.354845            1            1
    Mancini         D            Roma            11              0.351092            1            1
Cancellieri         A           Lazio             3              0.347858            1           

In [9]:
logger.info("\n=== Affaroni individuati (basato su regole fisse) ===")
old_display_columns = ['Nome', f"R_{suffix_current_year}", f"Squadra_{suffix_current_year}", f"Qt.A_{suffix_current_year}", f"Fm_{suffix_current_year}", f"Pv_{suffix_current_year}", "Indice_Beccalossi", "Fenomeno", "Affarone"]
df_affaroni = loaded_dataset[loaded_dataset['Affarone'] == 1]
if not df_affaroni.empty:
    df_affaroni_sorted = df_affaroni.sort_values(by='Indice_Beccalossi', ascending=False)
    logger.info(df_affaroni_sorted[old_display_columns].head(10).to_string(index=False))
else:
    logger.warning("Nessun giocatore identificato come 'Affarone' in base ai criteri attuali.")

2025-08-30 13:17:28.769 | INFO     | __main__:<module>:1 - 
=== Affaroni individuati (basato su regole fisse) ===
2025-08-30 13:17:28.784 | INFO     | __main__:<module>:6 -                Nome R_2025_26 Squadra_2025_26  Qt.A_2025_26  Fm_2025_26  Pv_2025_26  Indice_Beccalossi  Fenomeno  Affarone
             Bianco         C      Fiorentina             1         0.0           0           0.691988         0         1
         Vasquez D.         P            Roma             1         0.0           0           0.496093         0         1
Milinkovic-Savic V.         P          Napoli             4         0.0           0           0.467429         0         1
        Cancellieri         A           Lazio             3         5.5           1           0.445972         1         1
            Darmian         D           Inter             4         0.0           0           0.421414         1         1
        Terracciano         P           Milan             1         0.0           0      